In [1]:
import re
import pandas as pd
import math
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt

def is_a_string(x):
        return str(x).isdigit()

file_location = "WELLCOME_APCspend2013_forThinkful.csv"

#added encoding='latin1' to prevent utf-8 error 
df = pd.read_csv(file_location, header=0, quotechar='"', encoding='latin1')
df.columns.values[4] = "Cost"
df.dropna(inplace=True)
df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,Cost
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78
10,PMCID: PMC3739413,ACS (Amercian Chemical Society) Publications,Journal of Chemical Information and Modeling,Locating Sweet Spots for Screening Hits and Ev...,£1329.69


In [2]:
df['PMID/PMCID'].value_counts()

Not yet available                                   7
-                                                   7
In Process                                          7
Pub July 2013                                       3
Epub ahead of print - june 2013                     3
Epub ahead of print pub Jan 2013, print in press    3
print in press                                      3
pub Aug 2013                                        3
Epub ahead of print pub Feb 2013, print in press    3
PMC 3435256                                         2
PMC 3413714                                         2
PMC3529206                                          2
PMC3405234                                          2
PMC in progress                                     2
Monograph chapter, not in PMC                       2
PMC 3599138                                         2
41609                                               2
e-pub                                               2
PMC3597274                  

Using .value_counts we can see what some of the issues in our data are. For one, there are far too many letters that pollute the data to the point in which even similar PMC IDs are different. Thus to make it easier it'd be best if we just did away with all the letters within the PMC ID column. There are some variables that we can confirm:
1. PMC IDs only consist of integers
2. PMC IDs have the length of 7

In [3]:
#From this data we can see a variable of issues with the data. 
#Conditions for it to be a PMID/PMCID...Only numbers (no letters), 7 digits long
#------Failed Attempts
##df['PMID/PMCID'].apply(lambda x: re.sub('^[0-9]*', '', x)) #Get rid of all letters
##df['PMID/PMCID'].apply(lambda x: x.strip) #Get rid of all white spaces

df['PMID/PMCID'] = df['PMID/PMCID'].apply(lambda x: ''.join(list(filter(str.isdigit, str(x))))) #retrieve only the integers
df['PMID/PMCID'] = df['PMID/PMCID'].apply(lambda x: x if len(x) == 7 else (x[-7:] if len(x) > 7 else '')) #if the amount of digits isn't 7 then set to NaN
df['PMID/PMCID'].value_counts()

           82
3413714     2
3746156     2
3647051     2
3597274     2
3173209     2
3435256     2
2735079     2
3528370     2
3401426     2
3599138     2
3708033     2
3381227     2
3676342     2
3599488     2
3613719     2
3405234     2
3529206     2
3462035     1
3510437     1
3791410     1
3927070     1
3486906     1
3610885     1
3709123     1
3560059     1
3607403     1
3654172     1
3682181     1
3782194     1
           ..
3798095     1
3707360     1
3539357     1
3752052     1
3585959     1
3763373     1
3736666     1
3677912     1
3661540     1
3624797     1
3381722     1
3534503     1
3716626     1
3590645     1
3437796     1
3554572     1
2843621     1
3652741     1
3624963     1
3746110     1
3550523     1
3724654     1
3713434     1
3734352     1
3549497     1
3525566     1
3636241     1
3636136     1
3569615     1
3540259     1
Name: PMID/PMCID, Length: 1830, dtype: int64

In [4]:
df['Publisher'].value_counts()

Elsevier                                                       344
Public Library of Science                                      265
Wiley                                                          110
Oxford University Press                                         73
Springer                                                        72
Wiley-Blackwell                                                 55
ASBMB                                                           45
OUP                                                             45
Nature Publishing Group                                         44
BioMed Central                                                  40
BMC                                                             23
Nature                                                          23
Royal Society                                                   21
BMJ                                                             18
American Society for Biochemistry and Molecular Biology       

For publishers, I'd like to find the first letter of each word (seperated by spaces), and return  
If the amount of words is greater then 1, then it is not a abbreviation and then can be made into an abbreviation

Retrieve only the capital letters of each word to make into an abbreviation  (gets rid of the 'of', the, and, for)

In [5]:
def abbreviate(x):
    x = x.upper() #capitalize everything
    answer = ''
    for i in x.split():
        if i == 'OF' or i == 'AND' or i == 'THE' or i == '&' or i == 'FOR':
            pass
        else:
            if i[0] == '(' or i[0] == '/':
                pass
            else:
                answer = answer + i[0]
    return answer
        
df['Publisher'] = df['Publisher'].apply(lambda x: x.upper() if len(x.split()) == 1 else abbreviate(x))
df['Publisher'].value_counts()

ELSEVIER                348
PLS                     269
OUP                     135
WILEY                   110
SPRINGER                 72
ASBMB                    67
BC                       60
WILEY-BLACKWELL          55
NPG                      49
RS                       28
CUP                      24
PLOS                     24
ASM                      23
NATURE                   23
BMC                      23
CB                       22
ACS                      21
BMJ                      20
DJS                      19
ES                       17
FRONTIERS                17
WSSI                     16
NAS                      16
SGM                      16
LB                       15
OJ                       15
TF                       14
APA                      14
BG                       13
SAGE                     10
                       ... 
MP                        1
HPI                       1
CADMUS                    1
ASBMBI                    1
SGC                 

In [6]:
#Same as Publisher
df['Journal title'] = df['Journal title'].replace(['PLOSONE', 'PLOS ONE', 'PLOS 1', 'PLOS'], 'PLOS ONE') 
df['Journal title'] = df['Journal title'].replace(['ACTA D', 'ACTA CRYSTALLOGRAPHICA SECTION D', 'ACTA CRYSTALLOGRAPHY D', 'ACTA CRYSTALLOGRAPHICA, SECTION D', 'ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY'], 'ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY') 
df['Journal title'] = df['Journal title'].replace(['AMERICAN JNL EPIDEMIOLOGY'], 'AMERICAN JOURNAL OF EPIDEMIOLOGY') 
df['Journal title'] = df['Journal title'].replace(['AMERICAN JOURNAL OF MEDICAL GENETICS PART A'], 'AMERICAN JOURNAL OF MEDICAL GENETICS') 
df['Journal title'] = df['Journal title'].replace(['ANTIMICROBIAL AGENTS AND CHEMOTHERAPY', 'ANTIMICROBIAL AGFENTS AND CHEMOTHERAPY'], 'ANTIMICROBIAL AGENTS & CHEMOTHERAPY') 
df['Journal title'] = df['Journal title'].replace(['ANGEWANDE CHEMIE', 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION','ANGEW CHEMS INT ED' ], 'ANGEWANDTE CHEMIE') 
df['Journal title'] = df['Journal title'].replace(['BEHAVIOUR RESEARCH AND THERAPY'], 'BEHAVIOR RESEARCH & THERAPY')
df['Journal title'] = df['Journal title'].apply(lambda x: x.upper() if len(x.split()) == 1 else abbreviate(x))
df['Journal title'].value_counts()

PO                    190
JBC                    70
JN                     30
PNAS                   29
NAR                    28
NEUROIMAGE             28
PP                     24
PG                     23
PNTD                   22
BO                     21
NC                     19
BJ                     18
HMG                    17
FJ                     16
CR                     16
CC                     16
JID                    14
PM                     14
BPH                    13
JP                     13
CB                     13
JMC                    13
MD                     13
BRAIN                  11
JGV                    11
BJP                    11
JV                     10
MJ                     10
DC                     10
AN                     10
                     ... 
JHBS                    1
NEUROINFORMATICS        1
PU                      1
NI                      1
PTRSLBBS                1
GIM                     1
BT                      1
AJ          

In [7]:
def nosign(x):
    b=""
    b = str(x).replace("£", '')
    return b

df['Cost'] = df['Cost'].apply(lambda x: nosign(x))
df['Cost'].value_counts()

2040.00      90
999999.00    40
1500.00      26
2100.00      26
2400.00      25
1800.00      24
1700.00      20
3000.00      20
1834.77      18
825.68       18
852.00       15
1680.00      15
2034.00      12
2010.24      11
3120.00      11
1620.00      10
1920.00      10
1704.00       8
1260.00       8
1536.00       8
2340.00       8
1939.95       6
1044.00       6
1836.92       6
2625.60       6
2520.00       6
2184.22       6
2377.65       6
1068.00       6
3780.00       5
             ..
122.31        1
2424.03       1
409.78        1
1871.85       1
1152.50       1
1999.08       1
1173.00       1
1853.00       1
711.11        1
3021.72       1
3716.09       1
2561.40       1
2979.00       1
619.83        1
953.48        1
1283.76       1
1018.00       1
1974.72       1
2265.88       1
1860.37       1
889.97        1
1225.26       1
1728.80       1
773.45        1
1524.08       1
1577.90       1
2326.43       1
3838.88       1
1929.35       1
1343.39       1
Name: Cost, Length: 1295